In [1]:
import pandas as pd

In [2]:
f = pd.read_csv("dwbfgreyhoundwin05122022.csv")

In [3]:
f = f.iloc[:, :8]
f.drop(columns=["EVENT_ID", "SELECTION_ID"], inplace=True)

In [4]:
allowed_locs = ["AUS", "NZL"]
for t in enumerate(f.loc[:, "MENU_HINT"]):
    location = t[1].split(' / ')[0]
    if location not in allowed_locs:
        f.drop(t[0], inplace=True)

In [5]:
def get_date(event_dt):
    return event_dt.split()[0]

def get_track_loc(menu_hint):
    return menu_hint.split()[2]

def get_race_no(event_name):
    return int(event_name.split()[0][1:])

def get_dist(event_name):
    return int(event_name.split()[1][:-1])

def get_name(selection_name):
    return selection_name[3:]

In [6]:
f[["MENU_HINT", "EVENT_DT", "EVENT_NAME", "SELECTION_NAME"]] = f.transform({
    "MENU_HINT": get_track_loc, 
    "EVENT_DT": get_date, 
    "EVENT_NAME": get_race_no,
    "SELECTION_NAME": get_name
    })

In [24]:
f.rename(columns={
    "MENU_HINT": "track_loc",
    "EVENT_DT": "date",
    "EVENT_NAME": "race_no",
    "SELECTION_NAME": "dog_id",
    "WIN_LOSE": "win",
    "BSP": "bsp"
}, inplace=True)

In [49]:
def find_races(cleaned_df: pd.DataFrame, track_loc=None, date=None, \
    race_no=None, dog_id=None, win=None, bsp_filter=None):
    query = []
    if track_loc != None:
        query.append("track_loc == @track_loc")
    if date != None:
        query.append("date == @date")
    if race_no != None:
        query.append("race_no == @race_no")
    if dog_id != None:
        query.append("dog_id == @dog_id")
    if win != None:
        query.append("win == @win")
    if bsp_filter != None:
        query.append("bsp " + bsp_filter)
        
    return cleaned_df.query(" and ".join(query))